# Imports

In [15]:
import config
import requests

In [2]:
import riotwatcher
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Helper Functions

In [3]:
def check_rate(rate, limit):
    rate +=1
    if rate > limit-10:
        time.sleep(120)
        rate = 0
    return rate

In [4]:
def scrape_player_data(size):
    """
    Scrapes from an intial summoner, gathering summoner puuIDs in a set (to avoid duplicates)
    until set is equal to size in size.
    
    LOGIC:
    GET INIT
    GET MATCH
    GET PARTICIPANTS
    ADD UNIQUE PARTICIPANTS TO UNIQUE_CHAIN
    ADD PARTICIPANTS TO SUMMONER_SET
    FROM UNIQUE PARTICIPANTS GET MATCH
    GET PARTICIPANTS
    """
    

    #Sets up API
    tft = riotwatcher.TftWatcher(config.api_key)
    #GET INIT
    initial_summoner = tft.summoner.by_name('na1', config.initial_summoner)
    
    #Return object
    summoner_list = [initial_summoner['puuid']]
    
    while(len(summoner_list) < size):
        
        #counts number of requests made to API
        #limit is 100/2min
        rate = 0
        
        #Current Summoner is puuID
        current_summoner = summoner_list[-1]
        current_match_id_list = tft.match.by_puuid('americas',current_summoner, count=2)
        
        rate = check_rate(rate, 200)
        
        for current_match_id in current_match_id_list:
            current_match = tft.match.by_id('americas',current_match_id)
            
            rate = check_rate(rate,200)
            
            for participant_puuid in current_match['metadata']['participants']:
                #New puuID
                if participant_puuid not in summoner_list:
                    pass
    
    return summoner_list

In [23]:
def scrape_match_data(puuid_list, matches_per_player, verbose=True):
    
    """
    Taking in a list of puuid's, scrape_match_data retrieves the [matches_per_player] most recent matches.
    Stores them all in an empty DataFrame
    And Returns DataFrame
    """
    
    res = pd.DataFrame()
    rate = 0
    
    for puuid in puuid_list:
        
        try:
            match_list = tft.match.by_puuid('americas', puuid, count=matches_per_player)
            rate = check_rate(rate,200)

            for match_id in match_list:

                match=tft.match.by_id('americas', match_id)
                rate =check_rate(rate,200)

                match_df = pd.concat({match_id:pd.DataFrame(match['info']['participants'])}, names=['MATCH_ID'])

                res = pd.concat([res,match_df])

                if res.shape[0] % 1000 == 0:
                    if verbose:
                        print("*" * 30)
                        print(f'Reached {res.shape[0]} Rows')
                        print('Saving...')
                        print(f'Current Time: [{time.strftime("%H:%M:", time.localtime())}]')
                    res.to_csv('../datasets/match_df.csv', index_label=['MATCH_ID','index'])
        except requests.exceptions.HTTPError as exception:
            if verbose:
                print('*' * 30)
                print(f'Recieved Error: {exception.response.status_code}')
                print(f'Waiting 5 Minutes Before Continuing')
                print(f'Current Time: [{time.strftime("%H:%M:", time.localtime())}]')
            time.sleep(300)
            
    return res

# API/Wrapper Setup

In [18]:
print(f'Current Time {time.strftime("%H:%M:", time.localtime())}')

Current Time 12:24:42


In [6]:
tft = riotwatcher.TftWatcher(config.api_key)

# Data Acquisition

## Summoner Puuid Scraping

In [45]:
player_list = scrape_player_data(10_000) 

In [ ]:
player_list.to_csv('../datasets/summoner_puuids.csv',index=False)

## Match Scraping

In [7]:
player_list = pd.read_csv('../datasets/summoner_puuids.csv')

In [8]:
player_list.head()

,summoner_puuid
0,Kd4Xz5DXbrEphO15MDzByPFqYdWVMT_L27vH7Yq6DfDn66...
1,uYusCxO1ixEhomxcoiioMPfLebToRRcSKMt7xY39wE3V53...
2,RSs0bQs81xWzeL81iClAevfOirVGPBE4Ctf0lymmscxjSQ...
3,mBsMqJuSWqnxxLUGdPTiiHqDivZNrh64n3MND65hu--cV9...
4,Mm4mqIDNu7szCdtub6foYrxxIWtvKLI_Skkhev40_nwN2_...


In [ ]:
match_df = scrape_match_data(player_list['summoner_puuid'], 10)

******************************
Recieved Error: 503
Waiting 5 Minutes Before Continuing
Current Time: [13:02:]
******************************
Reached 1000 Rows
Saving...
Current Time: [13:08:]
******************************
Recieved Error: 503
Waiting 5 Minutes Before Continuing
Current Time: [13:10:]
******************************
Reached 2000 Rows
Saving...
Current Time: [13:15:]
******************************
Reached 3000 Rows
Saving...
Current Time: [13:19:]
******************************
Reached 4000 Rows
Saving...
Current Time: [13:22:]
******************************
Reached 5000 Rows
Saving...
Current Time: [13:26:]
******************************
Reached 6000 Rows
Saving...
Current Time: [13:28:]
******************************
Recieved Error: 503
Waiting 5 Minutes Before Continuing
Current Time: [13:30:]
******************************
Reached 7000 Rows
Saving...
Current Time: [13:37:]
******************************
Reached 8000 Rows
Saving...
Current Time: [13:40:]
**************